In [12]:
import numpy as np

from evaluation.mc import *
from utils.misc import *
from policies import *
import gym
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict
from tqdm import tqdm
from evaluation.td import *

In [9]:
def sample_episode_lim(env, policy,limit=100):
    """
    A sampling routine. Given environment and a policy samples one episode and returns states, actions, rewards
    and dones from environment's step function and policy's sample_action function as lists.

    Args:
        env: OpenAI gym environment.
        policy: A policy which allows us to sample actions with its sample_action method.

    Returns:
        Tuple of lists (states, actions, rewards, dones). All lists should have same length.
        Hint: Do not include the state after the termination in the list of states.
    """
    states = []
    actions = []
    rewards = []
    dones = []

    state = env.reset()

    for i in range(limit):
        states.append(state)

        action = policy.sample_action(state)
        state, reward, done, _ = env.step(action)

        actions.append(action)
        rewards.append(reward)
        dones.append(done)
        if done == True:
            break

    return states, actions, rewards, dones

In [10]:
import gym
from gym import spaces
from gym.utils import seeding

class NChainEnv(gym.Env):
    """n-Chain environment
    This game presents moves along a linear chain of states, with two actions:
     0) forward, which moves along the chain but returns no reward
     1) backward, which returns to the beginning and has a small reward
    The end of the chain, however, presents a large reward, and by moving
    'forward' at the end of the chain this large reward can be repeated.
    At each action, there is a small probability that the agent 'slips' and the
    opposite transition is instead taken.
    The observed state is the current state in the chain (0 to n-1).
    This environment is described in section 6.1 of:
    A Bayesian Framework for Reinforcement Learning by Malcolm Strens (2000)
    http://ceit.aut.ac.ir/~shiry/lecture/machine-learning/papers/BRL-2000.pdf
    """
    def __init__(self, n=5, slip=0.2, small=2, large=10):
        self.n = n
        self.slip = slip  # probability of 'slipping' an action
        self.small = small  # payout for 'backwards' action
        self.large = large  # payout at end of chain for 'forwards' action
        self.state = 0  # Start at beginning of the chain
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Discrete(self.n)
        self.seed()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        assert self.action_space.contains(action)
        
        if self.np_random.rand() < self.slip:
            action = not action  # agent slipped, reverse action taken
            
            
        #added 
        if self.state == 0 and action:
            done = True
        elif self.state == self.n - 1 and not action:
            done = True
        else:
            done = False
        #this part
            
        if action:  # 'backwards': go back to the beginning, get small reward
            reward = self.small
            self.state = 0
        elif self.state < self.n - 1:  # 'forwards': go up along the chain
            reward = 0
            self.state += 1
        else:  # 'forwards': stay at the end of the chain, collect large reward
            reward = self.large
        
        return self.state, reward, done, {}

    def reset(self):
        self.state = 0
        return self.state

# NChain

In [ ]:
import numpy as np
# from environments.Nchain import NChainEnv
from evaluation.mc import *
from utils.misc import *
from policies import *
import gym
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

for j in [5,10,15,20]:
    for i in range(10):
        n = j

        Q = np.zeros((n, 2))

        for state in range(n):
            Q[state,  0] = 1

        env = NChainEnv(n=n, slip = 0.0)
        actions = [0, 1]
        policy = EpsilonGreedyPolicy(actions, Q, 0.001)


        random_policy = RandomPolicy(actions)

        np.random.seed(i)
        _, hist = mc_prediction(env, policy, 500001, sample_episode, save_every=1000, name="mc_nchain_{}_{}".format(j,i))



 25%|██▌       | 125430/500001 [00:33<01:40, 3714.22it/s]

In [40]:
for j in [5,15,20]:
    for i in range(10):
        n = j

        Q = np.zeros((n, 2))

        for state in range(n):
            Q[state,  0] = 1

        env = NChainEnv(n=n, slip = 0.0)
        actions = [0, 1]
        policy = EpsilonGreedyPolicy(actions, Q, 0.001)


        random_policy = RandomPolicy(actions)

        np.random.seed(i)
        last, hist = mc_ordinary_importance_sampling(env, random_policy, policy, 500001, sample_episode, save_every=1000, name="mc_ordinary_nchain_{}_{}".format(j,i))

100%|██████████| 500001/500001 [01:29<00:00, 5615.93it/s]


In [41]:
from evaluation.td import *
for j in [5,15,20]:
    for i in range(10):
        n = j

        Q = np.zeros((n, 2))

        for state in range(n):
            Q[state,  0] = 1

        env = NChainEnv(n=n, slip = 0.0)
        actions = [0, 1]
        policy = EpsilonGreedyPolicy(actions, Q, 0.001)


        random_policy = RandomPolicy(actions)

        np.random.seed(i)
        _, V_hist = n_step_td_off_policy(env, random_policy, policy, 500001, sample_step, n=5, save_every=1000,name="td_nchain_{}_{}".format(j,i))

100%|██████████| 500001/500001 [02:55<00:00, 2855.57it/s]


In [45]:
for j in [5,15,20]:
    for i in range(10):
        n = j

        Q = np.zeros((n, 2))

        for state in range(n):
            Q[state,  0] = 1

        env = NChainEnv(n=n, slip = 0.0)
        actions = [0, 1]
        policy = EpsilonGreedyPolicy(actions, Q, 0.001)


        random_policy = RandomPolicy(actions)

        np.random.seed(i)
        last, hist = mc_weighted_importance_sampling(env, random_policy, policy, 500001, sample_episode, save_every=1000, name="mc_weighted_nchain_{}_{}".format(j,i))


100%|██████████| 500001/500001 [01:33<00:00, 5356.39it/s]


# BlackJack

In [42]:
for i in range(10):    
    player_values = [i for i in range(12, 22)]
    dealer_values = [i for i in range(1,11)]


    env = gym.make('Blackjack-v0')


    # Let's sample some episodes

    policy = SimpleBlackjackPolicy()
    actions = [0,1]
    blackjack_policy = RandomPolicy(actions)

    np.random.seed(i)

    last, hist = mc_ordinary_importance_sampling(env, blackjack_policy, SimpleBlackjackPolicy(), 500001, sample_episode, save_every=1000, name= "mc_ordinary_bj_"+str(i))

100%|██████████| 500001/500001 [01:18<00:00, 6339.37it/s]


In [43]:
for i in range(10):    
    player_values = [i for i in range(12, 22)]
    dealer_values = [i for i in range(1,11)]


    env = gym.make('Blackjack-v0')


    # Let's sample some episodes

    policy = SimpleBlackjackPolicy()
    actions = [0,1]
    blackjack_policy = RandomPolicy(actions)

    np.random.seed(i)
    _, hist = mc_prediction(env, SimpleBlackjackPolicy(), 500001, sample_episode, save_every=1000, name="mc_bj_"+str(i))

100%|██████████| 500001/500001 [01:18<00:00, 6368.10it/s]


In [44]:
for i in range(10):    
    player_values = [i for i in range(12, 22)]
    dealer_values = [i for i in range(1,11)]


    env = gym.make('Blackjack-v0')


    # Let's sample some episodes

    policy = SimpleBlackjackPolicy()
    actions = [0,1]
    blackjack_policy = RandomPolicy(actions)

    np.random.seed(i)
    _, V_hist = n_step_td_off_policy(env, blackjack_policy, SimpleBlackjackPolicy(), 500001, sample_step, n=5, save_every=1000, name="td_bj_"+str(i))

100%|██████████| 500001/500001 [01:35<00:00, 5246.30it/s]


In [46]:
for i in range(10):    
    player_values = [i for i in range(12, 22)]
    dealer_values = [i for i in range(1,11)]


    env = gym.make('Blackjack-v0')


    # Let's sample some episodes

    policy = SimpleBlackjackPolicy()
    actions = [0,1]
    blackjack_policy = RandomPolicy(actions)

    np.random.seed(i)
    _, hist = mc_weighted_importance_sampling(env, blackjack_policy, SimpleBlackjackPolicy(), 500001, sample_episode, save_every=1000, name= "mc_weighted_bj_"+str(i))


100%|██████████| 500001/500001 [01:18<00:00, 6409.93it/s]


# FrozenLake

In [47]:

from gym.envs.registration import register
register(
    id='FrozenLakeNotSlippery-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False},
    max_episode_steps=100,
    reward_threshold=0.78, # optimum = .8196
)

In [49]:
for i in range(10):
    env = gym.make('FrozenLakeNotSlippery-v0')
    env.is_slippery = False
    action_map = {
      0: "D",
      1: "R",
      2: "D",
      3: "L",
      4: "D",
      5: "U",
      6: "D",
      7: "U",
      8: "R",
      9: "D",
      10: "D",
      11: "U",
      12: "U",
      13: "R",
      14: "R",
      15: "D",

    }
    index = {"L": 0, "D": 1, "R": 2, "U": 3}

    Q = np.zeros((16, 4))
    for state, action in action_map.items():
        Q[state,  index[action]] =1

    actions = [0,1,2,3]
    target_policy =  EpsilonGreedyPolicy(actions, Q, 0.001)
    behavior_policy = RandomPolicy(actions)

    ### First we run the env with random agent

    np.random.seed(i)
    _, hist = mc_weighted_importance_sampling(env, behavior_policy, target_policy, 500001, sample_episode, save_every=1000, name= "mc_weighted_fl_"+str(i))



100%|██████████| 500001/500001 [03:40<00:00, 2268.90it/s]


In [50]:
for i in range(10):
    env = gym.make('FrozenLakeNotSlippery-v0')
    env.is_slippery = False
    action_map = {
      0: "D",
      1: "R",
      2: "D",
      3: "L",
      4: "D",
      5: "U",
      6: "D",
      7: "U",
      8: "R",
      9: "D",
      10: "D",
      11: "U",
      12: "U",
      13: "R",
      14: "R",
      15: "D",

    }
    index = {"L": 0, "D": 1, "R": 2, "U": 3}

    Q = np.zeros((16, 4))
    for state, action in action_map.items():
        Q[state,  index[action]] =1

    actions = [0,1,2,3]
    target_policy = EpsilonGreedyPolicy(actions, Q, 0.001)
    behavior_policy = RandomPolicy(actions)

    ### First we run the env with random agent

    np.random.seed(i)
    _, hist = mc_ordinary_importance_sampling(env, behavior_policy, target_policy, 500001, sample_episode, save_every=1000, name= "mc_ordinary_fl_"+str(i))




100%|██████████| 500001/500001 [03:36<00:00, 2306.24it/s]


In [16]:
for i in range(10):
    env = gym.make('FrozenLakeNotSlippery-v0')
    env.is_slippery = False
    action_map = {
      0: "D",
      1: "R",
      2: "D",
      3: "L",
      4: "D",
      5: "U",
      6: "D",
      7: "U",
      8: "R",
      9: "D",
      10: "D",
      11: "U",
      12: "U",
      13: "R",
      14: "R",
      15: "D",

    }
    index = {"L": 0, "D": 1, "R": 2, "U": 3}

    Q = np.zeros((16, 4))
    for state, action in action_map.items():
        Q[state,  index[action]] =1

    actions = [0,1,2,3]
    target_policy = EpsilonGreedyPolicy(actions, Q, 0.001)
    behavior_policy = RandomPolicy(actions)

    ### First we run the env with random agent

    np.random.seed(i)
    _, hist = n_step_td_off_policy(env, behavior_policy, target_policy, 500001, sample_step, save_every=1000, name= "td_fl_"+str(i))





100%|██████████| 500001/500001 [04:30<00:00, 1847.27it/s]


In [53]:
for i in range(10):
    env = gym.make('FrozenLakeNotSlippery-v0')
    env.is_slippery = False
    action_map = {
      0: "D",
      1: "R",
      2: "D",
      3: "L",
      4: "D",
      5: "U",
      6: "D",
      7: "U",
      8: "R",
      9: "D",
      10: "D",
      11: "U",
      12: "U",
      13: "R",
      14: "R",
      15: "D",

    }
    index = {"L": 0, "D": 1, "R": 2, "U": 3}

    Q = np.zeros((16, 4))
    for state, action in action_map.items():
        Q[state,  index[action]] =1

    actions = [0,1,2,3]
    target_policy = EpsilonGreedyPolicy(actions, Q, 0.001)
    behavior_policy = RandomPolicy(actions)

    ### First we run the env with random agent

    np.random.seed(i)
    _, hist = mc_prediction(env, target_policy, 500001, sample_episode, save_every=1000, name= "mc_fl_"+str(i))





100%|██████████| 500001/500001 [03:25<00:00, 2429.73it/s]
